Launch interactive version: 👉👉👉 [![Try ``dyce``](https://jupyterlite.readthedocs.io/en/latest/_static/badge.svg)](https://posita.github.io/dyce-notebooks/lab?path=github%2Fbumpity-posita-dyce-12%2Fbumpity.ipynb) 👈👈👈 *[[source](https://github.com/posita/dyce-notebooks/tree/main/notebooks/github/bumpity-posita-dyce-12)]*

## [``dyce``](https://posita.github.io/dyce/) solution to [“ Nth Die of 5D20 Plus Bump & Meta Die mechanic help”](https://github.com/posita/dyce/discussions/12)

Once viewing this notebook in Jupyter Lab, select ``Run All Cells`` from the ``Run`` menu above.

In [1]:
# Install additional requirements if necessary
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    try:
        import anydyce
    except (ImportError, ModuleNotFoundError):
        requirements = ["anydyce~=0.4.0", "icepool~=0.29"]
        try:
            import piplite ; await piplite.install(requirements)
            # Work around <https://github.com/jupyterlite/jupyterlite/issues/838>
            import matplotlib.pyplot ; matplotlib.pyplot.clf()
        except ImportError:
            import pip ; pip.main(["install"] + requirements)
    import anydyce

try:
    import showit
except ImportError:
    # Work-around for JupyterLite in non-Chromium browsers
    import js
    import os
    from urllib.parse import urljoin, urlparse, urlunparse
    loc_url = urlparse(js.location.toString())
    ext_root = loc_url.path.find("/extensions/@jupyterlite/")
    if ext_root < 0:
        base_url = urljoin(js.location.toString(), "../files/")
    else:
        loc_url = loc_url._replace(path=loc_url.path[:ext_root])
        base_url = urljoin(urlunparse(loc_url), "files/")
    for path in (
                "github/bumpity-pool-posita-dyce-12/dyce_impl.py",
                "github/bumpity-pool-posita-dyce-12/icepool_impl.py",
                "github/bumpity-pool-posita-dyce-12/params.py",
                "github/bumpity-pool-posita-dyce-12/showit.py",
            ):
        url = urljoin(base_url, path)
        res = await js.fetch(url)
        assert 200 <= res.status < 300
        text = await res.text()
        with open(os.path.basename(path), "w") as f:
            f.write(text)
    import showit

Code can be found in:

* [``dyce_impl.py``](dyce_impl.py) - primary implementation
* [``icepool_impl.py``](icepool_impl.py) [icepool](https://github.com/HighDiceRoller/icepool) variant
* [``params.py``](params.py) - parsing and parameter validation
* [``showit.py``](showit.py) - interactive UI

The UI accepts inputs (one per line) of the following format:

```
{M}s {N}b @{C} [ >{A} | <{D} ] [ +@{I} ... ] [ # {comment} ]
```

Whitespace is ignored. `{…}` indicates a variable. `[…]` indicates optional notation. `… | …` indicates a selection. `… ...` indicates repetition.

* `{M}` - integer > 0; the number of standard dice in the pool
* `{N}` - integer >= 0; the number of bump dice in the pool
* `{C}` - integer >= 1, <= total pool size; the one-based index of the set die
* `{A}` - integer >= 1; an optional scale of the advantage (mutually exclusive from `<{D}`)
* `{D}` - integer >= 1; an optional scale of the disadvantage (mutually exclusive from `>{A}`)
* `{I}` - integer >= 1; one or more optional one-based index(es) of any bonus(es)
* `{comment}` - string; an optional user comment

Examples:

* `3s2b@1` - use a pool of three standard dice and two bump dice, with the first (lowest) die as the set die
* `1s4b@5+@5` - use a pool of 1 standard die and four bump dice, with the fifth (highest) die as the set die, adding the fifth (highest) die as a bonus
* `1s4b@5<2` - use a pool of 1 standard die and four bump dice, with the fifth (highest) die as the set die, with two disadvantage dice
* `4s1b@2>1+@3+@3  # Whoo boy!` - use a pool of four standard dice and one bump die, with the second die as the set die, with one advantage die, adding the third die as a bonus (twice), with a comment of `Whoo boy!`

Unparseable lines are silently ignored.

In [2]:
from dyce import H
from showit import showit

die_map = {
    "d4": H(4),
    "d6": H(6),
    "d8": H(8),
    "d10": H(10),
    "d12": H(12),
    "d20": H(20),
    "d0": H({0: 1}),
    "d10*2": H(10) * 2,
}

notations = r"""
5s0b@3  # for stat 10
4s1b@3  # for stat 11
3s2b@3  # for stat 12

[d10*2]5s0b@3  # for stat 10 (d10*2)
[d10*2]4s1b@3  # for stat 11 (d10*2)
[d10*2]3s2b@3  # for stat 12 (d10*2)

2s3b@3  # for stat 13
1s4b@3  # for stat 14
[d0]1s0b@1  # spacer (ignore)

[d10*2]2s3b@3  # for stat 13 (d10*2)
[d10*2]1s4b@3  # for stat 14 (d10*2)
[d0]1s0b@1  # spacer (ignore)

[d6]2s3b@5>2+@3
"""

showit(notations, die_map, selected_die=die_map["d20"])

Output()